This notebook is aiming at prepare training data for second subgroup classifier of multi-step strategies. It also contains the part to kick off training jobs after data is prepared. (Training job will use "auto-split" mode to reserve validation data). Also, this training set includes the keyword records prepared in other notebook.

Prerequisite:

1) csv files containing extracted text files

2) A csv file prepared in Step2 notebook, containing label information corresponding to text files

3) Keyword records prepared in step2.5 notebook

Output:

1) Prepared training file for subgroup classifier

2) Kick off binary classification training job

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import boto3

In [2]:
# Load prior prepared extracted text files
textract_df = pd.read_csv("Textract_output_train04.csv")
textract_df.rename(columns={'file_name' : 'filename'}, inplace=True)
textract_df

filename  page  \
0                 1003700000000530903_11159719.pdf     1   
1                 1003700000000530903_11159719.pdf     2   
2                 1003700000000530903_11159719.pdf     3   
3                 1003700000000530903_11159719.pdf     4   
4                 1003700000000530903_11159719.pdf     5   
...                                            ...   ...   
840163  OCR_33049_1003700000000697103_11103804.pdf    43   
840164  OCR_33049_1003700000000697103_11103804.pdf    44   
840165  OCR_33049_1003700000000697103_11103804.pdf    45   
840166  OCR_33049_1003700000000697103_11103804.pdf    46   
840167  OCR_33049_1003700000000697103_11103804.pdf    47   

                                                  content  
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...  
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...  
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...  
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...  
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...  
...                                                   ...  
840163  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  
840164  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840165  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840166  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840167  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  

[840168 rows x 3 columns]

In [3]:
# Change filename to same format 
textract_df['filename'] = textract_df['filename'].str.split('_').str[-2] + '_' + textract_df['filename'].str.split('_').str[-1]

In [4]:
# Load prior prepared reformat label file
label_df = pd.read_csv("batch3_label_revised.csv")
label_df

page             class                          filename  filename_cut
0         29   admission_order  1003700000000633142_11068501.pdf        633142
1         43  history_physical  1003700000000633142_11068501.pdf        633142
2         44  history_physical  1003700000000633142_11068501.pdf        633142
3         45  history_physical  1003700000000633142_11068501.pdf        633142
4         46  history_physical  1003700000000633142_11068501.pdf        633142
...      ...               ...                               ...           ...
357778    81  history_physical  1003700000000675009_11251999.pdf        675009
357779    82  history_physical  1003700000000675009_11251999.pdf        675009
357780    83  history_physical  1003700000000675009_11251999.pdf        675009
357781    84  history_physical  1003700000000675009_11251999.pdf        675009
357782    85  history_physical  1003700000000675009_11251999.pdf        675009

[357783 rows x 4 columns]

In [5]:
# Change AO/AT/AR name to "admit_group"
label_df['class'] = label_df['class'].replace('admission_order', 'admit_group')
label_df['class'] = label_df['class'].replace('admit_type', 'admit_group')
label_df['class'] = label_df['class'].replace('admission_reason', 'admit_group')

In [6]:
# Change PR/PQ/DD/CM/NA name to "lower_group"
label_df['class'] = label_df['class'].replace('procedures', 'lower_group')
label_df['class'] = label_df['class'].replace('physician_query', 'lower_group')
label_df['class'] = label_df['class'].replace('discharge_disposition', 'lower_group')
label_df['class'] = label_df['class'].replace('case_management', 'lower_group')
label_df['class'] = label_df['class'].replace('nutritional_assessment', 'lower_group')

In [7]:
# Check distributions to see if it satisfies 10:1 ratio requirement
label_df['class'].value_counts()

flowsheets              97615
progress_notes          63926
mar                     46243
labs                    43658
admit_group             22026
consult                 18063
history_physical        15633
radiology               15390
emergency_department    14605
discharge_summary       13080
lower_group              7544
Name: class, dtype: int64

In [9]:
# Creating grouping class will introduce duplicate records. We need to remove the duplicate records.
label_df.drop_duplicates(inplace=True)
label_df

page             class                          filename  filename_cut
0         29       admit_group  1003700000000633142_11068501.pdf        633142
1         43  history_physical  1003700000000633142_11068501.pdf        633142
2         44  history_physical  1003700000000633142_11068501.pdf        633142
3         45  history_physical  1003700000000633142_11068501.pdf        633142
4         46  history_physical  1003700000000633142_11068501.pdf        633142
...      ...               ...                               ...           ...
357778    81  history_physical  1003700000000675009_11251999.pdf        675009
357779    82  history_physical  1003700000000675009_11251999.pdf        675009
357780    83  history_physical  1003700000000675009_11251999.pdf        675009
357781    84  history_physical  1003700000000675009_11251999.pdf        675009
357782    85  history_physical  1003700000000675009_11251999.pdf        675009

[354810 rows x 4 columns]

In [10]:
# Check common file
common_files = set(textract_df.filename).intersection(label_df.filename)
common_files_train = common_files
len(common_files_train)

2378

In [11]:
# filter out uncommon file in text dataframe
textract_df_train = textract_df[textract_df.filename.isin(common_files_train)]
textract_df_train

filename  page  \
0       1003700000000530903_11159719.pdf     1   
1       1003700000000530903_11159719.pdf     2   
2       1003700000000530903_11159719.pdf     3   
3       1003700000000530903_11159719.pdf     4   
4       1003700000000530903_11159719.pdf     5   
...                                  ...   ...   
840163  1003700000000697103_11103804.pdf    43   
840164  1003700000000697103_11103804.pdf    44   
840165  1003700000000697103_11103804.pdf    45   
840166  1003700000000697103_11103804.pdf    46   
840167  1003700000000697103_11103804.pdf    47   

                                                  content  
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...  
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...  
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...  
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...  
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...  
...                                                   ...  
840163  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  
840164  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840165  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840166  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840167  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  

[838573 rows x 3 columns]

In [12]:
# filter out uncommon file in label dataframe
label_df_train = label_df[label_df.filename.isin(common_files_train)]
label_df_train

page             class                          filename  filename_cut
0         29       admit_group  1003700000000633142_11068501.pdf        633142
1         43  history_physical  1003700000000633142_11068501.pdf        633142
2         44  history_physical  1003700000000633142_11068501.pdf        633142
3         45  history_physical  1003700000000633142_11068501.pdf        633142
4         46  history_physical  1003700000000633142_11068501.pdf        633142
...      ...               ...                               ...           ...
357778    81  history_physical  1003700000000675009_11251999.pdf        675009
357779    82  history_physical  1003700000000675009_11251999.pdf        675009
357780    83  history_physical  1003700000000675009_11251999.pdf        675009
357781    84  history_physical  1003700000000675009_11251999.pdf        675009
357782    85  history_physical  1003700000000675009_11251999.pdf        675009

[353684 rows x 4 columns]

In [13]:
# merge text files with labels
merged_file_train = pd.merge(textract_df_train, label_df_train, how = "left", on = ["filename", "page"])
merged_file_train

filename  page  \
0       1003700000000530903_11159719.pdf     1   
1       1003700000000530903_11159719.pdf     2   
2       1003700000000530903_11159719.pdf     3   
3       1003700000000530903_11159719.pdf     4   
4       1003700000000530903_11159719.pdf     5   
...                                  ...   ...   
856409  1003700000000697103_11103804.pdf    43   
856410  1003700000000697103_11103804.pdf    44   
856411  1003700000000697103_11103804.pdf    45   
856412  1003700000000697103_11103804.pdf    46   
856413  1003700000000697103_11103804.pdf    47   

                                                  content      class  \
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...        NaN   
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...        NaN   
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...        NaN   
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...        NaN   
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...        NaN   
...                                                   ...        ...   
856409  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  radiology   
856410  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  radiology   
856411  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  radiology   
856412  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  radiology   
856413  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  radiology   

        filename_cut  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN  
...              ...  
856409      697103.0  
856410      697103.0  
856411      697103.0  
856412      697103.0  
856413      697103.0  

[856414 rows x 5 columns]

In [14]:
# Drop unlabelled records as we already removed "others" class in first step
merged_file_train.dropna(subset=['class'], inplace=True)
merged_file_train

filename  page  \
15      1003700000000530903_11159719.pdf    16   
16      1003700000000530903_11159719.pdf    17   
17      1003700000000530903_11159719.pdf    18   
18      1003700000000530903_11159719.pdf    19   
19      1003700000000530903_11159719.pdf    20   
...                                  ...   ...   
856409  1003700000000697103_11103804.pdf    43   
856410  1003700000000697103_11103804.pdf    44   
856411  1003700000000697103_11103804.pdf    45   
856412  1003700000000697103_11103804.pdf    46   
856413  1003700000000697103_11103804.pdf    47   

                                                  content        class  \
15      University of Kentucky HospitalOptum Medical N...  admit_group   
16      UK Emar By Patient - HIMFormatCriteria for sel...          mar   
17      UK Emar By Patient - HIMFormatBAKER, EDWINNA T...          mar   
18      UK Emar By Patient - HIMFormatBAKER, EDWINNA T...          mar   
19      UK Emar By Patient - HIMFormatBAKER, EDWINNA T...          mar   
...                                                   ...          ...   
856409  Patient: GABRY, KAREN11800 E. Twelve Mile Road...    radiology   
856410  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...    radiology   
856411  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...    radiology   
856412  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...    radiology   
856413  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...    radiology   

        filename_cut  
15          530903.0  
16          530903.0  
17          530903.0  
18          530903.0  
19          530903.0  
...              ...  
856409      697103.0  
856410      697103.0  
856411      697103.0  
856412      697103.0  
856413      697103.0  

[353263 rows x 5 columns]

In [15]:
# Prepare data ready for "multi-label" mode
merged_file_train = merged_file_train[['content', 'class']].drop_duplicates().groupby('content')['class'].apply('|'.join).reset_index()

In [16]:
## Drop flowsheets records as the number of flowsheets is more than 10 times than lower_group

frac_num = 25000
merged_file_train = merged_file_train.drop(merged_file_train[merged_file_train['class']=='flowsheets'].sample(frac_num).index)

In [17]:
# Check distribution
merged_file_train['class'].value_counts()

flowsheets                                              70233
progress_notes                                          58213
mar                                                     45030
labs                                                    40373
consult                                                 15572
                                                        ...  
discharge_summary|emergency_department|admit_group          1
discharge_summary|labs|history_physical                     1
labs|lower_group|radiology                                  1
radiology|labs|emergency_department|history_physical        1
history_physical|lower_group|consult                        1
Name: class, Length: 418, dtype: int64

In [18]:
# Check training data ready for classifier
merged_file_train

content  \
0       !! UNABLE TO FIND 4 tablets every morning. Med...   
1       !! predniSONE 20 MG tabletTake 40mg of prednis...   
2       " John J Worch Jr. is a 76 y.o. male with a hi...   
3       " Ms. Grosnickle is an 84-year-old female with...   
4       " pt is a 68 y.o. Caucasian female with past m...   
...                                                   ...   
333858  |Injection atI Peak pharmacologic stress++Imag...   
333859  |Injection at| Peak pharmacologic stress+Image...   
333860  ~100' with RW and min WBAT R LE, with verbal c...   
333861  ~Department of Medical Records-Patient: DEVILL...   
333862  ~If ANY answer 'yes'- please consult Wound Car...   

                                     class  
0       admit_group|labs|discharge_summary  
1                        discharge_summary  
2               admit_group|progress_notes  
3                        discharge_summary  
4                                  consult  
...                                    ...  
333858                      progress_notes  
333859                      progress_notes  
333860                         lower_group  
333861       discharge_summary|admit_group  
333862                    history_physical  

[308863 rows x 2 columns]

In [19]:
# Add most recent keyword records (prepared in other notebooks)
keyword_file = "keywords_to_training_v2.csv"
keyword_df = pd.read_csv(keyword_file)
keyword_df

class                      content
0                admit_type  Admission status: Inpatient
1                admit_type    admission for observation
2                admit_type             admission orders
3                admit_type   admission status inpatient
4                admit_type       admission to inpatient
..                      ...                          ...
700  nutritional_assessment         Nutritional Problems
701  nutritional_assessment    nutritional re-assessment
702  nutritional_assessment        Reason for Assessment
703  nutritional_assessment     Service  Type: Nutrition
704  nutritional_assessment           service:nutrtition

[705 rows x 2 columns]

In [20]:
# Replace class name in keyword dataframe
keyword_df['class'] = keyword_df['class'].replace('admission_order', 'admit_group')
keyword_df['class'] = keyword_df['class'].replace('admit_type', 'admit_group')
keyword_df['class'] = keyword_df['class'].replace('admission_reason', 'admit_group')

In [21]:
# Replace class name in keyword dataframe
keyword_df['class'] = keyword_df['class'].replace('procedures', 'lower_group')
keyword_df['class'] = keyword_df['class'].replace('physician_query', 'lower_group')
keyword_df['class'] = keyword_df['class'].replace('discharge_disposition', 'lower_group')
keyword_df['class'] = keyword_df['class'].replace('case_management', 'lower_group')
keyword_df['class'] = keyword_df['class'].replace('nutritional_assessment', 'lower_group')

In [22]:
# Concatenate training records with keyword records
merged_file_train = pd.concat([merged_file_train, keyword_df], axis=0, ignore_index = True)
merged_file_train

content  \
0       !! UNABLE TO FIND 4 tablets every morning. Med...   
1       !! predniSONE 20 MG tabletTake 40mg of prednis...   
2       " John J Worch Jr. is a 76 y.o. male with a hi...   
3       " Ms. Grosnickle is an 84-year-old female with...   
4       " pt is a 68 y.o. Caucasian female with past m...   
...                                                   ...   
309563                               Nutritional Problems   
309564                          nutritional re-assessment   
309565                              Reason for Assessment   
309566                           Service  Type: Nutrition   
309567                                 service:nutrtition   

                                     class  
0       admit_group|labs|discharge_summary  
1                        discharge_summary  
2               admit_group|progress_notes  
3                        discharge_summary  
4                                  consult  
...                                    ...  
309563                         lower_group  
309564                         lower_group  
309565                         lower_group  
309566                         lower_group  
309567                         lower_group  

[309568 rows x 2 columns]

In [23]:
# Reorder the training dataframe to satisfy Comprehend's requirement
merged_file_train = merged_file_train[['class', 'content']]

In [25]:
# Save training data locally
merged_file_train.to_csv('multistep-2-subgroup-training-data-autosplit-keyword.csv', index=False, header=False)

In [26]:
# Specify region and client
REGION = 'us-east-1'
s3 = boto3.resource('s3')
comprehend = boto3.client('comprehend', region_name = REGION)

In [27]:
# upload training file to S3 (we're uploading to a folder that training pipeline is not monitoring, so we can manually kickoff jobs)
s3.Bucket('scio-saas-datastores').upload_file('multistep-2-subgroup-training-data-autosplit-keyword.csv', 'SaaS_workspace/aws_comprehend/inputs_manual/multistep-2-subgroup-training-data-autosplit-keyword.csv')

In [28]:
# Specify training parameters (classifier name has to be UNIQUE)
name = 'multistep-2-custom-classifier-subgroup-yy-autosplit-keyword'
language_code = 'en'
training_bucket = 'scio-saas-datastores'
training_key = 'SaaS_workspace/aws_comprehend/inputs_manual/multistep-2-subgroup-training-data-autosplit-keyword.csv'
data_access_role_arn = 'arn:aws:iam::097079433570:role/service-role/AmazonComprehendServiceRole-aws_scio_S3Datastore_servicewrite'

In [29]:
# Kick off training job
response = comprehend.create_document_classifier(
        DocumentClassifierName = name,
        LanguageCode = language_code,
        InputDataConfig = {
            "DataFormat": "COMPREHEND_CSV",
            "S3Uri":f"s3://{training_bucket}/{training_key}"
        },

        OutputDataConfig = {
            "S3Uri":"s3://scio-saas-datastores/SaaS_workspace/aws_comprehend/output/batch3-multistep",
            "KmsKeyId": "arn:aws:kms:us-east-1:097079433570:key/1c6d057c-0be3-40a6-b23c-5870a92b901f"
        },

        DataAccessRoleArn = data_access_role_arn,
        VolumeKmsKeyId = "arn:aws:kms:us-east-1:097079433570:key/1c6d057c-0be3-40a6-b23c-5870a92b901f",
        Mode = "MULTI_LABEL",

)